In [201]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression

In [202]:
enem_test = pd.read_csv("test.csv", sep=',', index_col=False)
enem_train = pd.read_csv("train.csv",sep=',', index_col=False)

In [203]:
enem.columns

Index(['NU_INSCRICAO', 'CO_MUNICIPIO_RESIDENCIA', 'NO_MUNICIPIO_RESIDENCIA',
       'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA', 'NU_IDADE', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE',
       ...
       'Q041', 'Q042', 'Q043', 'Q044', 'Q045', 'Q046', 'Q047', 'Q048', 'Q049',
       'Q050'],
      dtype='object', length=165)

In [204]:
# Remove useless columns
enem_train.drop(['Unnamed: 0', 'NU_ANO'], axis=1, inplace=True)

In [205]:
enem.shape

(13730, 165)

In [206]:
enem_train.describe()

,CO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,NU_IDADE,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,...,Q032,Q033,Q034,Q035,Q036,Q037,Q038,Q039,Q040,Q041
count,1.373000e+04,13730.000000,13730.000000,13202.000000,13730.000000,13730.000000,1.312100e+04,13121.000000,13730.000000,13730.000000,...,6354.000000,6354.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,2938.000000
mean,3.088548e+06,30.736708,21.593809,0.116043,2.185652,1.048871,3.057946e+06,30.429159,1.782666,2.146176,...,4.530217,3.962858,4.645594,4.678441,3.341515,4.458995,3.724618,3.506409,4.442098,3.671545
std,9.890495e+05,9.862405,7.043654,0.365986,1.007285,0.258895,9.703020e+05,9.670030,0.880734,3.217328,...,1.027613,1.625356,0.920998,0.865039,1.711930,1.200172,1.689362,1.718858,1.181549,1.746444
min,1.100023e+06,11.000000,13.000000,0.000000,0.000000,0.000000,1.100023e+06,11.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.307650e+06,23.000000,17.000000,0.000000,1.000000,1.000000,2.306900e+06,23.000000,1.000000,0.000000,...,5.000000,3.000000,5.000000,5.000000,2.000000,5.000000,3.000000,2.000000,4.000000,3.000000
50%,3.118601e+06,31.000000,19.000000,0.000000,3.000000,1.000000,3.106200e+06,31.000000,2.000000,0.000000,...,5.000000,5.000000,5.000000,5.000000,4.000000,5.000000,5.000000,4.000000,5.000000,5.000000
75%,3.550308e+06,35.000000,23.000000,0.000000,3.000000,1.000000,3.549805e+06,35.000000,2.000000,3.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
max,5.300108e+06,53.000000,67.000000,3.000000,5.000000,4.000000,5.300108e+06,53.000000,4.000000,10.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [207]:
aux_dt = pd.DataFrame({'columns': enem_train.columns,
                       'type': enem_train.dtypes,
                       'nulls': enem_train.isnull().sum()})
aux_dt.sort_values('nulls', ascending=False)

,columns,type,nulls
SG_UF_ENTIDADE_CERTIFICACAO,SG_UF_ENTIDADE_CERTIFICACAO,object,12092
NO_ENTIDADE_CERTIFICACAO,NO_ENTIDADE_CERTIFICACAO,object,12092
CO_UF_ENTIDADE_CERTIFICACAO,CO_UF_ENTIDADE_CERTIFICACAO,float64,12092
Q041,Q041,float64,10792
CO_ESCOLA,CO_ESCOLA,float64,9448
...,...,...,...
IN_MOBILIARIO_OBESO,IN_MOBILIARIO_OBESO,int64,0
IN_LAMINA_OVERLAY,IN_LAMINA_OVERLAY,int64,0
IN_PROTETOR_AURICULAR,IN_PROTETOR_AURICULAR,int64,0
IN_MEDIDOR_GLICOSE,IN_MEDIDOR_GLICOSE,int64,0


In [208]:
corr = enem_train.corr().abs().sort_values('NU_NOTA_MT', ascending=False)

In [251]:
# variáveis que possuem maior correlação com a nota de matemática
features = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC']

In [252]:
enem_train[features].isnull().sum()

NU_NOTA_CN    0
NU_NOTA_CH    0
NU_NOTA_LC    0
dtype: int64

In [253]:
enem_train['NU_NOTA_CN'] = enem_train['NU_NOTA_CN'].fillna(0)
enem_train['NU_NOTA_CH'] = enem_train['NU_NOTA_CH'].fillna(0)
enem_train['NU_NOTA_LC'] = enem_train['NU_NOTA_LC'].fillna(0)
enem_train['NU_NOTA_MT'] = enem_train['NU_NOTA_MT'].fillna(0)

In [254]:
enem_test[features].isnull().sum()

NU_NOTA_CN    0
NU_NOTA_CH    0
NU_NOTA_LC    0
dtype: int64

In [255]:
enem_test['NU_NOTA_CN'] = enem_test['NU_NOTA_CN'].fillna(0)
enem_test['NU_NOTA_CH'] = enem_test['NU_NOTA_CH'].fillna(0)
enem_test['NU_NOTA_LC'] = enem_test['NU_NOTA_LC'].fillna(0)

In [256]:
# Selecione a inscrição do teste e a variável dependente (nota de matemática do dataset de treino)
inscricao = enem_test['NU_INSCRICAO']
y = enem_train['NU_NOTA_MT']

In [257]:
new_train_dataset = enem_train[features]
new_test_dataset = enem_test[features]

In [278]:
scaler = preprocessing.StandardScaler()

In [279]:
train_scaled_df = scaler.fit_transform(new_train_dataset)
test_scaled_df = scaler.fit_transform(new_test_dataset)

In [280]:
train_scaled_df = pd.DataFrame(train_scaled_df, columns=features)
test_scaled_df = pd.DataFrame(test_scaled_df, columns=features)

In [281]:
train_scaled_df.head()

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC
0,0.373596,0.406787,0.852512
1,0.552708,0.612129,0.928373
2,-1.672124,-1.682045,-1.624476
3,-1.672124,-1.682045,-1.624476
4,-1.672124,-1.682045,-1.624476


In [282]:
model = LinearRegression()
model.fit(train_scaled_df, y)
r_sq = model.score(train_scaled_df, y)
r_sq

0.9010821003848966

In [283]:
y_pred = model.predict(test_scaled_df)

In [285]:
result = pd.DataFrame(list(zip(inscricao,y_pred)),columns=['NU_INSCRICAO','NU_NOTA_MT'])
result.describe()

,NU_NOTA_MT
count,4576.000000
mean,356.092607
std,217.172969
min,-3.744913
25%,90.684973
50%,451.049973
75%,503.381949
max,698.494624


In [286]:
result.to_csv('answer.csv',encoding='UTF-8',index=False)